In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
if SETTING["signal_type"] == "raw":

    raise

In [ ]:
feature_context = pd.read_csv(
    PATH["feature_x_sample.feature_context.tsv"], sep="\t", index_col=0
)

feature_context.columns.name = SETTING["sample_alias"]

In [ ]:
sample_context = pd.read_csv(
    PATH["feature_x_sample.sample_context.tsv"], sep="\t", index_col=0
).T

sample_context.index.name = SETTING["feature_alias"]

In [ ]:
feature_context_negative = -feature_context.clip(upper=0)

feature_context_positive = feature_context.clip(lower=0)

In [ ]:
sample_context_negative = -sample_context.clip(upper=0)

sample_context_positive = sample_context.clip(lower=0)

In [ ]:
if (
    "feature" in SETTING["context_element_types"]
    and "sample" in SETTING["context_element_types"]
):

    normalization_method = "0-1"

    context_negative = ccal.normalize_s_or_df(
        feature_context_negative, 1, normalization_method
    ) + ccal.normalize_s_or_df(sample_context_negative, 0, normalization_method)

    context_positive = ccal.normalize_s_or_df(
        feature_context_positive, 1, normalization_method
    ) + ccal.normalize_s_or_df(sample_context_positive, 0, normalization_method)

elif "feature" in SETTING["context_element_types"]:

    context_negative = feature_context_negative

    context_positive = feature_context_positive

elif "sample" in SETTING["context_element_types"]:

    context_negative = sample_context_negative

    context_positive = sample_context_positive

In [ ]:
if SETTING["select_feature_automatically"]:

    standard_deviation = SETTING["select_feature_standard_deviation"]

    title = {"text": context_negative.index.name}

    xaxis = {"title": "Rank"}

    context_negative_features = ccal.select_series_indices(
        context_negative.sum(axis=1),
        ">",
        standard_deviation=standard_deviation,
        title=title,
        xaxis=xaxis,
        yaxis={"title": "{} Signal Sum".format("Negative")},
    )

    context_positive_features = ccal.select_series_indices(
        context_positive.sum(axis=1),
        ">",
        standard_deviation=standard_deviation,
        title=title,
        xaxis=xaxis,
        yaxis={"title": "{} Signal Sum".format("Positive")},
    )

else:

    context_negative_features = (
        context_negative.replace(0, np.nan).dropna(how="all").index
    )

    context_positive_features = (
        context_positive.replace(0, np.nan).dropna(how="all").index
    )

context_negative_features.size, context_positive_features.size

In [ ]:
if SETTING["select_sample_automatically"]:

    standard_deviation = SETTING["select_sample_standard_deviation"]

    title = {"text": context_negative.columns.name}

    xaxis = {"title": "Rank"}

    context_negative_samples = ccal.select_series_indices(
        context_negative.sum(axis=1),
        ">",
        standard_deviation=standard_deviation,
        title=title,
        xaxis=xaxis,
        yaxis={"title": "{} Signal Sum".format("Negative")},
    )

    context_positive_samples = ccal.select_series_indices(
        context_positive.sum(axis=1),
        ">",
        standard_deviation=standard_deviation,
        title=title,
        xaxis=xaxis,
        yaxis={"title": "{} Signal Sum".format("Positive")},
    )

else:

    context_negative_samples = (
        context_negative.replace(0, np.nan).dropna(how="all").columns
    )

    context_positive_samples = (
        context_positive.replace(0, np.nan).dropna(how="all").columns
    )

context_negative_samples.size, context_positive_samples.size

In [ ]:
context_negative_signal = context_negative.loc[
    context_negative_features, context_negative_samples
]

In [ ]:
context_positive_signal = context_positive.loc[
    context_positive_features, context_positive_samples
]

In [ ]:
if "negative" in SETTING["contexts"] and "positive" in SETTING["contexts"]:

    context_negative_signal.index = pd.Index(
        ("(-) {}".format(feature) for feature in context_negative_signal.index),
        name=context_negative_signal.index.name,
    )

    context_positive_signal.index = pd.Index(
        ("(+) {}".format(feature) for feature in context_positive_signal.index),
        name=context_positive_signal.index.name,
    )

    context_signal = pd.concat((context_negative_signal, context_positive_signal))

elif "negative" in SETTING["contexts"]:

    context_signal = context_negative_signal

elif "positive" in SETTING["contexts"]:

    context_signal = context_positive_signal

In [ ]:
if SETTING["signal_normalization_method"] is not None:

    context_signal = ccal.normalize_s_or_df(
        context_signal,
        SETTING["signal_normalization_axis"],
        SETTING["signal_normalization_method"],
    )

In [ ]:
context_signal = ccal.clean_and_write_df_to_tsv(
    context_signal, context_signal.index.name, PATH["feature_x_sample.signal.tsv"]
)

ccal.summarize_feature_x_sample(
    context_signal,
    feature_x_sample_alias="Context Signal",
    feature_x_sample_value_name="Context Signal",
    plot_heat_map_max_size=SETTING["plot_heat_map_max_size"],
    plot_histogram_max_size=SETTING["plot_histogram_max_size"],
    plot_rug_max_size=SETTING["plot_rug_max_size"],
)